# Билет _

# Задание 1

# Задание 2

<img src="img/11_2.png" width=800>

In [2]:
import sqlite3


In [25]:
def get_all_playlists(cur: sqlite3.Cursor) -> list:
    stmt = 'SELECT * FROM Playlist'
    return cur.execute(stmt).fetchall()


def get_playlist_n_tracks_sum_milliseconds(cur: sqlite3.Cursor, playlist_name: str) -> list:
    stmt = '''
    SELECT COUNT(T.TrackId), SUM(T.Milliseconds)
    FROM Playlist P
    JOIN PlaylistTrack PT ON P.PlaylistId = PT.PlaylistId
    JOIN Track T ON PT.TrackId = T.TrackId
    WHERE P.Name = ? GROUP BY P.PlaylistId;
    '''
    return cur.execute(stmt, [playlist_name]).fetchall()

In [26]:
con = sqlite3.connect('data/task_2/Chinook_Sqlite.sqlite')
cur = con.cursor()

In [27]:
get_all_playlists(cur)

[(1, 'Music'),
 (2, 'Movies'),
 (3, 'TV Shows'),
 (4, 'Audiobooks'),
 (5, '90’s Music'),
 (6, 'Audiobooks'),
 (7, 'Movies'),
 (8, 'Music'),
 (9, 'Music Videos'),
 (10, 'TV Shows'),
 (11, 'Brazilian Music'),
 (12, 'Classical'),
 (13, 'Classical 101 - Deep Cuts'),
 (14, 'Classical 101 - Next Steps'),
 (15, 'Classical 101 - The Basics'),
 (16, 'Grunge'),
 (17, 'Heavy Metal Classic'),
 (18, 'On-The-Go 1')]

In [28]:
get_playlist_n_tracks_sum_milliseconds(cur, 'Music')

[(3290, 877683083), (3290, 877683083)]

# Задание 3

<img src="img/11_3.png" width=800>

In [236]:
import math
import numpy as np
import multiprocessing as mp

In [237]:
def foo(ints_sample):
    limit = 0.99999999
    result = []

    n = len(ints_sample)
    seq = list(range(n))
    for i in seq:
        left = ints_sample[i]
        for j in seq[i:]:
            right = ints_sample[j]
            sin_mul = math.sin(left * right)
            if sin_mul > limit:
                result.append((left, right, sin_mul))

    return result

In [238]:
%%file "src/task_11.py"
import math


def foo(chunk, ints_sample):
    limit = 0.99999999
    result = set()
    for i in chunk:
        for j in ints_sample:
            sin_mul = math.sin(i * j)
            if sin_mul > limit:
                result.add((frozenset([i, j]), sin_mul))

    return result

Overwriting src/task_11.py


In [239]:
def blocking_mode(ints_sample):
    return foo(ints_sample)


def nonblocking_mode(ints_sample):
    from src.task_11 import foo

    def chunkinize(lst, n: int):
        n = len(lst) // n
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    n = mp.cpu_count()
    chunks = list(chunkinize(ints_sample, n))
    with mp.Pool(n) as pool:
        res = pool.starmap(foo, zip(chunks, [ints_sample] * n))
        return list({i for chunk in res for i in chunk})

In [240]:
ints_sample = np.arange(1, 3000 + 1)

In [241]:
blocking_mode(ints_sample)

[(23, 2253, 0.999999999696513),
 (57, 2746, 0.9999999998635031),
 (62, 2536, 0.9999999970500172),
 (69, 751, 0.999999999696513),
 (91, 2855, 0.9999999980220771),
 (114, 1373, 0.9999999998635031),
 (115, 2253, 0.9999999924128237),
 (124, 1268, 0.9999999970500172),
 (129, 2025, 0.9999999983363994),
 (133, 2746, 0.99999999925685),
 (135, 1935, 0.9999999983363994),
 (157, 1006, 0.9999999906018032),
 (158, 2316, 0.9999999951152019),
 (193, 1896, 0.9999999951152019),
 (194, 2951, 0.9999999982003109),
 (209, 2746, 0.9999999981648745),
 (215, 1215, 0.9999999983363994),
 (219, 2149, 0.9999999901999105),
 (222, 2582, 0.9999999999999568),
 (225, 1161, 0.9999999983363994),
 (227, 2522, 0.9999999982003109),
 (237, 1544, 0.9999999951152019),
 (241, 2807, 0.9999999968696842),
 (243, 1075, 0.9999999983363994),
 (248, 634, 0.9999999970500172),
 (266, 1373, 0.99999999925685),
 (267, 2531, 0.9999999902820274),
 (269, 1739, 0.9999999948839151),
 (271, 2880, 0.999999995173126),
 (285, 2746, 0.9999999965875

In [242]:
nonblocking_mode(ints_sample)

[(frozenset({1492, 1782}), 0.999999994998317),
 (frozenset({2219, 2841}), 0.9999999961980565),
 (frozenset({602, 1991}), 0.9999999999814237),
 (frozenset({753, 2007}), 0.9999999999595361),
 (frozenset({215, 1215}), 0.9999999983363994),
 (frozenset({1551, 2049}), 0.9999999984365575),
 (frozenset({409, 1910}), 0.9999999992793377),
 (frozenset({1726, 1781}), 0.9999999993232765),
 (frozenset({1336, 2769}), 0.9999999995198774),
 (frozenset({1525, 1537}), 0.9999999964929694),
 (frozenset({91, 2855}), 0.9999999980220771),
 (frozenset({443, 2704}), 0.9999999985315368),
 (frozenset({719, 1668}), 0.9999999977965826),
 (frozenset({1039, 1556}), 0.9999999939010542),
 (frozenset({288, 2710}), 0.999999995173126),
 (frozenset({1442, 2276}), 0.9999999971839069),
 (frozenset({586, 2754}), 0.9999999914573485),
 (frozenset({542, 1440}), 0.999999995173126),
 (frozenset({1598, 2968}), 0.999999996866661),
 (frozenset({129, 2025}), 0.9999999983363994),
 (frozenset({285, 2746}), 0.9999999965875765),
 (frozens

In [243]:
%%timeit

blocking_mode(ints_sample)

1.45 s ± 63.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [244]:
%%timeit

nonblocking_mode(ints_sample)

1.08 s ± 70.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
